In [ ]:
pip install pyaudio

In [ ]:
pip install SpeechRecognition

In [ ]:
import cv2
import pyttsx3
import speech_recognition as sr
import time
import threading
import os

# Set the directory to save the captured image
save_directory = r"C:\Users\Manan Computer\Desktop\face capture system"
os.makedirs(save_directory, exist_ok=True)  # Ensure the directory exists

# Initialize text-to-speech engine
engine = pyttsx3.init()
engine.setProperty('rate', 135)  # Calm speech rate
engine.setProperty('volume', 0.9)  # Soft tone

# Load Face Detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start video capture with high resolution
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)  # Set width to 1920 pixels
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)  # Set height to 1080 pixels

# Flag to track face detection
face_detected = False

def recognize_speech():
    """Enhanced speech recognition system with noise reduction & retries."""
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Adjusting for background noise...")
        recognizer.adjust_for_ambient_noise(source, duration=1.5)  # Better noise reduction
        print("Listening for speech...")
        
        for _ in range(3):  # Allow up to 3 attempts
            try:
                audio = recognizer.listen(source, timeout=5)  # Timeout prevents hanging
                response = recognizer.recognize_google(audio).lower()
                print("User said:", response)
                return response
            except sr.UnknownValueError:
                print("Could not understand, trying again...")
            except sr.RequestError:
                print("Speech service unavailable, trying again...")
            except sr.WaitTimeoutError:
                print("No speech detected, trying again...")

        print("Failed to recognize speech after 3 attempts.")
        return None  # Return None if no successful recognition

def handle_face_detection(frame):
    """Handles face detection and initiates interaction."""
    global face_detected, cap
    if not face_detected:
        face_detected = True
        
        # Improve image quality
        frame = cv2.GaussianBlur(frame, (3, 3), 0)  # Reduce noise
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        
        # Apply contrast enhancement
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        enhanced_gray = clahe.apply(gray)
        frame = cv2.cvtColor(enhanced_gray, cv2.COLOR_GRAY2BGR)
        
        # Save the best-quality image
        image_path = os.path.join(save_directory, "detected_face.jpg")
        cv2.imwrite(image_path, frame, [int(cv2.IMWRITE_JPEG_QUALITY), 100])  # High-quality save
        print(f"Photo captured and saved as {image_path}")
        
        # Release the camera immediately after detecting a face
        cap.release()
        cv2.destroyAllWindows()
        
        engine.say("Aslam-ul-Allakium, Sheryar! It's always nice to see you.")
        engine.runAndWait()
        time.sleep(1)
        
        engine.say("How are you feeling today?")
        engine.runAndWait()
        
        # Start speech response in a new thread
        speech_thread = threading.Thread(target=process_speech_response)
        speech_thread.start()

def process_speech_response():
    """Processes the user's speech response and responds accordingly."""
    user_response = recognize_speech()
    
    if user_response:
        positive_responses = ["fine", "good", "okay", "great", "happy", "excellent"]
        negative_responses = ["not fine", "bad", "sad", "depressed", "terrible", "upset"]
        
        if any(word in user_response for word in positive_responses):
            engine.say("That’s wonderful to hear, Sheryar! Stay happy and take care.")
            time.sleep(1)
            engine.say("Have a great day. Goodbye!")
            engine.runAndWait()
        
        elif any(word in user_response for word in negative_responses):
            engine.say("Oh no, I’m really sorry to hear that, Sheryar.")
            time.sleep(1)
            engine.say("Just remember, you're not alone. I'm always here for you.")
            time.sleep(1)
            engine.say("Take care of yourself, Sheryar. Goodbye!")
            engine.runAndWait()
        
        else:
            engine.say("I'm not sure I understood, but I hope you're doing well, Sheryar.")
            engine.runAndWait()
    
    # End the program after the interaction
    exit()

# Capture multiple frames to select the best one
best_frame = None
max_variance = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50))

    if len(faces) > 0:
        # Calculate sharpness (Laplacian variance)
        variance = cv2.Laplacian(gray, cv2.CV_64F).var()
        
        if variance > max_variance:  # Pick the sharpest frame
            max_variance = variance
            best_frame = frame.copy()

        if len(faces) > 0 and best_frame is not None:
            handle_face_detection(best_frame)
            break  # Exit loop after detecting and saving the best face

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
